In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import csv

In [2]:
baseurl = "http://www.darklyrics.com/i/ironmaiden.html"
soup = bs(requests.get(baseurl).text)

In [3]:
song_link_selector = 'div.album a'
song_links = soup.select(song_link_selector)

In [4]:
album_soups = {}
for link in song_links:
    album = link.get_attribute_list('href')[0].split('/')[-1]
    album = re.sub('\.html#\d+', '', album)
    if album not in album_soups:
        alblink = f'http://www.darklyrics.com/lyrics/ironmaiden/{album}.html#1'
        albsoup = bs(requests.get(alblink).text)
        album_soups[album] = albsoup

In [5]:
title_year = album_soups['ironmaiden'].select_one('h2').text
title, year = re.findall('\"(.+)\".+\((\d+)\)', title_year)[0]
year = int(year)
title, year

('Iron Maiden', 1980)

In [6]:
iron = album_soups['ironmaiden']
iron_lyrics = iron.select_one('div.lyrics')

In [7]:
title = None
iron_songs = {}
for child in iron_lyrics.children:
    if child.name == 'h3':
        title = re.sub('\d+\.', '', child.text).strip()
        continue
    if title and not hasattr(child, 'text'):
        text = child.strip()
        if text:
            iron_songs.setdefault(title, [])
            iron_songs[title].append(text)

In [8]:
iron_songs.keys()

dict_keys(['Prowler', 'Sanctuary', 'Remember Tomorrow', 'Running Free', 'Phantom Of The Opera', 'Strange World', 'Charlotte The Harlot', 'Iron Maiden'])

In [30]:
iron_songs['Prowler']

['Walking through the city, looking oh so pretty,',
 "I've just got to find my way.",
 'See the ladies flashing. All their legs and lashes.',
 "I've just got to find my way.",
 'Well you see me crawling through the bushes with it open wide.',
 'What you seeing girl?',
 "Can't you believe that feeling, can't you believe it,",
 "Can't you believe your eyes?",
 "It's the real thing girl.",
 'Got me feeling myself and reeling around,',
 'Got me talking but feel like walking around.',
 'Got me feeling myself and reeling a...',
 "Got me talking but nothing's with me...",
 'Got me feeling myself and reeling around.']

In [36]:
songdata = []
songs_seen = set()
for albsoup in album_soups.values():
    title = None
    this_song = {}
    lyrics = None
    title_year = albsoup.select_one('h2').text
    album_title, year = re.findall('\"(.+)\".+\((\d+)\)', title_year)[0]
    year = int(year)
    song_num = None
    song = None
    all_lyrics = albsoup.select_one('div.lyrics')
    for child in all_lyrics.children:
        if child.name == 'h3':
            if song:
                songdata.append(song)
            title = re.sub('\d+\.', '', child.text).strip()
            song_num = int(re.findall('^\d+',  child.text)[0])
            stripped_title = re.sub('\(.+\)', '', title).strip()
            # gets rid of things like "Public Enema Number One (Live)"
            # by converting them to the base song ("Public Enema Number One")
            if stripped_title in songs_seen:
                song = None
            else:
                songs_seen.add(title)
                song = {
                    'album': album_title,
                    'year': year,
                    'num': song_num,
                    'title': title,
                    'lyrics': '',
                }
            continue
        if not song:
            continue
        
        if title and not hasattr(child, 'text'):
            text = child.strip()
            if text:
                song['lyrics'] += text + '\\n'
    if song:
        songdata.append(song)

In [37]:
'Public Enema Number One (Live)' in songs_seen

False

In [38]:
len(songdata)

187

In [39]:
len(songs_seen)

187

In [40]:
print(songdata[8])

{'album': 'Iron Maiden', 'year': 1980, 'num': 9, 'title': 'Iron Maiden', 'lyrics': "Won't you come into my room, I wanna show you all my wares.\\nI just want to see your blood, I just want to stand and stare.\\nSee the blood begin to flow as it falls upon the floor.\\nIron Maiden can't be fought, Iron Maiden can't be sought.\\nOh Well, wherever, wherever you are,\\nIron Maiden's gonna get you, no matter how far.\\nSee the blood flow watching it shed up above my head.\\nIron Maiden wants you for dead.\\nWon't you come into my room, I wanna show you all my wares.\\nI just want to see your blood, I just want to stand and stare.\\nSee the blood begin to flow as it falls upon the floor.\\nIron Maiden can't be fought, Iron Maiden can't be sought.\\n"}


In [34]:
songs_seen

{'2 A.M.',
 '2 Minutes To Midnight',
 '22 Acacia Avenue',
 'Aces High',
 'Afraid To Shoot Strangers',
 'Age Of Innocence',
 'Age Of Innocence... How Old?',
 'Alexander The Great',
 'Another Life',
 'Back In The Village',
 'Be Quick Or Be Dead',
 'Black Bart Blues',
 'Blood Brothers',
 "Blood On The World's Hands",
 'Brave New World',
 'Brighter Than A Thousand Suns',
 'Bring Your Daughter... To The Slaughter',
 'Burning Ambition',
 'Can I Play With Madness',
 'Can I Play With Madness?',
 'Caught Somewhere In Time',
 'Chains Of Misery',
 'Charlotte The Harlot',
 "Childhood's End",
 'Children Of The Damned',
 'Coming Home',
 'Como Estais Amigos',
 'Dance Of Death',
 'Darkest Hour',
 'Days Of Future Past',
 'Death Of The Celts',
 'Death Or Glory',
 'Deja-Vu',
 'Die With Your Boots On',
 'Different World',
 'Doctor, Doctor',
 "Don't Look To The Eyes Of A Stranger",
 'Dream Of Mirrors',
 'Drifter',
 'El Dorado',
 'Empire Of The Clouds',
 'Face In The Sand',
 'Fates Warning',
 'Fear Is The K

In [41]:
any('\t' in song['lyrics'] for song in songdata)

False

In [42]:
with open('iron_maiden_songs.tsv', 'w', encoding='utf-8') as f:
    f.write('\t'.join(songdata[0].keys()) + '\n')
    for song in songdata:
        if 'Ã©' in song['title']:
            song['title'] = song['title'].replace('Ã©', 'é')
            song['lyrics'] = song['lyrics'].replace('Ã©', 'é')
        f.write('\t'.join(str(x) for x in song.values()) + '\n')